In [8]:
import credentials
import twitter_settings  
import imp 
imp.reload(twitter_settings)
imp.reload(credentials)
from twitter_settings import *
import re
import tweepy
import mysql.connector
import pandas as pd
import numpy as np
#from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json


- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
- negative sentiment: compound score <= -0.05

In [9]:
class MyStreamListener(tweepy.StreamListener):
    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")
        
    def on_status(self,status):
  
        print("Tweet started")
        
        print('Retweet status: {0}'.format(status.retweeted))
        print('Original tweet: {0}'.format(status.text))
        
        if status.retweeted:
            ## for now avoid retweeted info
            return True

        #Extract attributes of each tweet
        id_str=status.id_str
        created_at=status.created_at
        print("Created at: {0}".format(created_at))
        original_text=status.text
        text=deEmojify_tweet(status.text)
        text=clean_tweet(text)

        print('Clean tweet: {0}'.format(text))
        
        ## use Vader to check sentiment of text
        ########################################
        analyser = SentimentIntensityAnalyzer()
        
        score = analyser.polarity_scores(status.text)
        print("{:-<40} \n {}".format(text, str(score)))
        
        if score['compound']>=0.05: #positive
            polarity = 1
        elif (score['compound']>-0.05) & (score['compound']<0.05): #neutral
            polarity = 0
        elif score['compound']<=-0.05: #negative
            polarity = -1
            
        compound_score=score['compound']
        positive_score=score['pos']
        negative_score=score['neg']
        neutral_score=score['neu']
        print('polarity: {0}'.format(polarity))
        
        ## set subjectivity as 0 for Vader
        subjectivity = 0
        ###########################################
        
        ## use TextBlob to check sentiment of text
        ########################################
#         sentiment=TextBlob(text).sentiment
#         polarity=sentiment.polarity
#         subjectivity=sentiment.subjectivity
        ######################################
        
       
        
        hashtags = []   #make an empty list

        for hashtag in status.entities['hashtags']:    #iterate over the list
            hashtags.append(hashtag["text"])             #append each hashtag to 'hashtags'

        print('hashtags: {0}'.format(str(hashtags)))
        #usermentions_list = json.dump(user_mentions)
        user_id = status.user.id_str
        user_name = status.user.name
        user_screen_name = status.user.screen_name
        print('userId: {0}, userName: {1}, userScreenName: {2}'.format(user_id,user_name,user_screen_name))
        user_created_at = status.user.created_at
        user_location=deEmojify_tweet(status.user.location)
        user_description=deEmojify_tweet(status.user.description)
        
        user_friends_count = status.user.friends_count
        user_followers_count=status.user.followers_count
        user_following = status.user.following
        print('user follower count: {0}, userFollowing: {1}'.format(user_followers_count,user_following))
        print('user friends count: {0}'.format(user_friends_count))
        
        #------------------------------------------
        tweetFound=False
        track_val = ''
        tweet_found_count = 0
        
        netflix_check_list = ['Netflix','netflix']
        netflix_check_set = set(netflix_check_list)
#         appletv_check_list = ['AppleTV','Apple TV','appletv','apple tv']
#         appletv_check_set = set(appletv_check_list)
#         disney_check_list = ['DisneyPlus','Disney+','disneyplus','disney+','disney','Disney','Disneyplus']
#         disney_check_set = set(disney_check_list)
#         amazon_check_list = ['Amazon Prime Video','amazon prime video','AmazonPrimeVideo',
#                      'amazonprimevideo','AmazonPrime Video','Amazon Prime','amazon prime','amazon','prime',
#                     'Amazon','Prime']
#         amazon_check_set = set(amazon_check_list)
# #         hbo_check_list = ['HBOMax','HBO Max','hbomax']
# #         hbo_check_set = set(hbo_check_list)

        text_set = set(text.split())
#         if amazon_check_set.intersection(text_set):
#             track_val = 'AmazonPrime'
#             tweetFound=True
#             tweet_found_count = tweet_found_count + 1
#         if appletv_check_set.intersection(text_set):
#             track_val = 'AppleTV'
#             tweetFound=True
#             tweet_found_count = tweet_found_count + 1
#         if disney_check_set.intersection(text_set):
#             track_val = 'DisneyPlus'
#             tweetFound=True
#             tweet_found_count = tweet_found_count + 1
# #         if hbo_check_set.intersection(text_set):
# #             track_val = 'HBOmax'
# #             tweetFound=True
# #             tweet_found_count = tweet_found_count + 1
        if netflix_check_set.intersection(text_set):
            track_val = 'Netflix'
            tweetFound=True
            tweet_found_count = tweet_found_count + 1
        
        if (tweet_found_count > 1):
            tweetFound = False
        
        print("Lets begin to record! All the best :)")
        
        longitude=None
        latitude=None
        if status.coordinates:
            longitude=status.coordinates['coordinates'][0]
            latitude=status.coordinates['coordinates'][1]
            
        retweet_count=status.retweet_count
        favorite_count=status.favorite_count
        
        
        
        print('latitude: {0}  logitude: {1}'.format(latitude,longitude))
        
        
        if tweetFound:
            if mydb.is_connected():
                mycursor=mydb.cursor()

                sql = "INSERT IGNORE INTO {} (id_str, created_at, track ,text, original_text, polarity, compound_score, negative_score, positive_score, neutral_score,subjectivity, hashtags,\
                user_created_at, user_id, user_name, user_screen_name,\
                user_location, user_description, user_followers_count, user_friends_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s ,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(twitter_settings.TABLE_NAME)
                val = (id_str, created_at, track_val, text, original_text, polarity, compound_score, negative_score, positive_score, neutral_score ,subjectivity, str(hashtags), user_created_at, user_id, user_name, user_screen_name, user_location, \
                    user_description, user_followers_count, user_friends_count, longitude, latitude, retweet_count, favorite_count)
                mycursor.execute(sql, val)
                mydb.commit()
                mycursor.close()
        
    def on_error(self, status_code):
        
        ##Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        
        if status_code == 420:
            # return False to disconnect the stream
            return False

In [10]:
def clean_tweet(tweet): 
     
    ##Use sumple regex statemnents to clean tweet text by removing links and special characters
    
    tweet_text= ' '.join(re.sub("(RT @[\w_]+:)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
    tweet_text = re.sub(r'(.)\1{2,}',r'\1\1',tweet_text)
    return tweet_text

def deEmojify_tweet(text):
    
    ##Strip all non-ASCII characters to remove emoji characters
    
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [11]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="*******",
    database="TwitterDB",
    auth_plugin='mysql_native_password',
    charset='utf8'
    
)
if mydb.is_connected():
    mycursor = mydb.cursor()
    mycursor.execute("SELECT COUNT(*) FROM information_schema.tables where table_name='{0}'".
                     format(twitter_settings.TABLE_NAME))
if mycursor.fetchone()[0] != 1:
    mycursor.execute("CREATE TABLE {} ({})".format(twitter_settings.TABLE_NAME, twitter_settings.TABLE_ATTRIBUTES))
    mydb.commit()
    mycursor.close()

In [12]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [13]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = twitter_settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()


You are now connected to the streaming API.
Tweet started
Retweet status: False
Original tweet: Two weeks in a row now I’ve tried to watch #JackRyan  but I think @PrimeVideo needs more servers - constant  buffer… https://t.co/9QRzuBnAY9
Created at: 2019-12-03 02:49:42
Clean tweet: Two weeks in a row now Ive tried to watch JackRyan but I think needs more servers constant buffer
Two weeks in a row now Ive tried to watch JackRyan but I think needs more servers constant buffer 
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
polarity: 0
hashtags: ['JackRyan']
userId: 708973, userName: petulantMarmot, userScreenName: tendancer
user follower count: 304, userFollowing: False
user friends count: 98
Lets begin to record! All the best :)
latitude: None  logitude: None
Tweet started
Retweet status: False
Original tweet: RT @AwaeYou: Ok, real talk, I’m a tv buff. I love stranger things, outlander, the crown, etc. But you could cancel any one of those shows a…
Created at: 2019-12-03 02:49:44

Tweet started
Retweet status: False
Original tweet: Dolly Parton’s Heartstrings on Netflix is a must 🌻🦋😢💓
Created at: 2019-12-03 02:49:52
Clean tweet: Dolly Partons Heartstrings on Netflix is a must
Dolly Partons Heartstrings on Netflix is a must 
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
polarity: 0
hashtags: []
userId: 1111404135239335936, userName: kaylin elizabeth, userScreenName: kaylinp_32
user follower count: 34, userFollowing: False
user friends count: 83
Lets begin to record! All the best :)
latitude: None  logitude: None
Tweet started
Retweet status: False
Original tweet: ‘In the dark’ on Netflix is such a good ass show.
Created at: 2019-12-03 02:49:52
Clean tweet: In the dark on Netflix is such a good ass show
In the dark on Netflix is such a good ass show 
 {'neg': 0.243, 'neu': 0.556, 'pos': 0.201, 'compound': -0.1531}
polarity: -1
hashtags: []
userId: 2404994355, userName: Jenaè, userScreenName: jenaelovee
user follower count: 239, userFollowing: False
user f

Tweet started
Retweet status: False
Original tweet: RT @goodbeanalt: shes laying in his king sized bed, watching Netflix. she sees your text. she ignores it and puts her phone on do not distu…
Created at: 2019-12-03 02:49:57
Clean tweet: shes laying in his king sized bed watching Netflix she sees your text she ignores it and puts her phone on do not distu
shes laying in his king sized bed watching Netflix she sees your text she ignores it and puts her phone on do not distu 
 {'neg': 0.077, 'neu': 0.923, 'pos': 0.0, 'compound': -0.2732}
polarity: -1
hashtags: []
userId: 3040037459, userName: Abby Curry🌙☮️, userScreenName: abbyismyname_8
user follower count: 226, userFollowing: False
user friends count: 93
Lets begin to record! All the best :)
latitude: None  logitude: None
Tweet started
Retweet status: False
Original tweet: RT @ignacio_aries: what i want to watch:

⚪sunset
⚪netflix
🔘you while smiling
Created at: 2019-12-03 02:49:57
Clean tweet: what i want to watch sunset netflix you wh

Tweet started
Retweet status: False
Original tweet: these posts are a lot funnier when you the nigga she ignoring the texts for lmao
Created at: 2019-12-03 02:50:04
Clean tweet: these posts are a lot funnier when you the nigga she ignoring the texts for lmao
these posts are a lot funnier when you the nigga she ignoring the texts for lmao 
 {'neg': 0.127, 'neu': 0.563, 'pos': 0.31, 'compound': 0.5994}
polarity: 1
hashtags: []
userId: 1085717805310402561, userName: beastboy 🦖, userScreenName: samidioooo
user follower count: 97, userFollowing: False
user friends count: 257
Lets begin to record! All the best :)
latitude: None  logitude: None
Tweet started
Retweet status: False
Original tweet: Just discovered season 2 of High Seas is on Netflix. I know what I’m doing tonight! https://t.co/91fZwfHd5l
Created at: 2019-12-03 02:50:04
Clean tweet: Just discovered season 2 of High Seas is on Netflix I know what Im doing tonight
Just discovered season 2 of High Seas is on Netflix I know what Im d

Tweet started
Retweet status: False
Original tweet: RT @dayumiumz: SELLING
👉spotify
2mos P60
3mos P80
renew 3mos P110 
8mos 250 new / 280 renew

👉netflix
1month 
solo 190pesos
shared 90pesos…
Created at: 2019-12-03 02:50:10
Clean tweet: SELLING spotify 2mos P60 3mos P80 renew 3mos P110 8mos 250 new 280 renew netflix 1month solo 190pesos shared 90pesos
SELLING spotify 2mos P60 3mos P80 renew 3mos P110 8mos 250 new 280 renew netflix 1month solo 190pesos shared 90pesos 
 {'neg': 0.0, 'neu': 0.897, 'pos': 0.103, 'compound': 0.34}
polarity: 1
hashtags: []
userId: 1201339781415784449, userName: RILAKKUMA, userScreenName: iamrilakkuma__
user follower count: 150, userFollowing: False
user friends count: 437
Lets begin to record! All the best :)
latitude: None  logitude: None
Tweet started
Retweet status: False
Original tweet: RT @IsraelUSAforevr: RETWEET if you will NOT be watching Barack and Michelle Obama’s Netflix series!
Created at: 2019-12-03 02:50:10
Clean tweet: RETWEET if you will NOT 

...

Tweet started
Retweet status: False
Original tweet: RT @exokai_kr: 191202 NETFLIX 6UNDERGROUND World Premiere OBSESSION EXO KAI

#EXOKAI #KAI #엑소카이 #카이 ⁠   
#EXO⁠ ⁠ #weareoneEXO⁠ ⁠ #엑소 https:…
Created at: 2019-12-03 03:45:15
Clean tweet: 191202 NETFLIX 6UNDERGROUND World Premiere OBSESSION EXO KAI EXOKAI KAI EXO weareoneEXO https
191202 NETFLIX 6UNDERGROUND World Premiere OBSESSION EXO KAI EXOKAI KAI EXO weareoneEXO https 
 {'neg': 0.156, 'neu': 0.844, 'pos': 0.0, 'compound': -0.4824}
polarity: -1
hashtags: ['EXOKAI', 'KAI', '엑소카이', '카이', 'EXO', 'weareoneEXO', '엑소']
userId: 947326761951412224, userName: 𝟴, userScreenName: forpalpal
user follower count: 397, userFollowing: False
user friends count: 139
Lets begin to record! All the best :)
latitude: None  logitude: None
Tweet started
Retweet status: False
Original tweet: RT @kindredsnows: CBC, Netflix and Northwood playing with us  https://t.co/mQ2mZlCRjE
Created at: 2019-12-03 03:45:16
Clean tweet: CBC Netflix and Northwood playing wit

Tweet started
Retweet status: False
Original tweet: RT @InqEnt: #ryanreynolds #AdriaArjona #MelanieLaurent trying to “shake their groove thing” after #EXO’s song &amp; dance number @InqEnt @Upper…
Created at: 2019-12-03 03:45:26
Clean tweet: ryanreynolds AdriaArjona MelanieLaurent trying to shake their groove thing after EXOs song amp dance number
ryanreynolds AdriaArjona MelanieLaurent trying to shake their groove thing after EXOs song amp dance number 
 {'neg': 0.0, 'neu': 0.933, 'pos': 0.067, 'compound': 0.0772}
polarity: 1
hashtags: ['ryanreynolds', 'AdriaArjona', 'MelanieLaurent', 'EXO']
userId: 1007570463264931842, userName: MY 2ND COMEBACK WITH EXO!!!! #OBSESSIONWITHEXO, userScreenName: dohdokyungsoo
user follower count: 34, userFollowing: False
user friends count: 150
Lets begin to record! All the best :)
latitude: None  logitude: None
Tweet started
Retweet status: False
Original tweet: RT @BBW_BFF: PSA: The Long Kiss Goodnight, one of my favorite movies ever, is on Netflix now

Tweet started
Retweet status: False
Original tweet: @Gleamapp is giving away a smart TV bundle and I want to Netflix all day this holiday season. Help me out 🙏 https://t.co/Tkdy1gY9oH
Created at: 2019-12-03 03:45:33
Clean tweet: is giving away a smart TV bundle and I want to Netflix all day this holiday season Help me out
is giving away a smart TV bundle and I want to Netflix all day this holiday season Help me out 
 {'neg': 0.0, 'neu': 0.551, 'pos': 0.449, 'compound': 0.8814}
polarity: 1
hashtags: []
userId: 77126491, userName: charles parker, userScreenName: parkedcar
user follower count: 132, userFollowing: False
user friends count: 1718
Lets begin to record! All the best :)
latitude: None  logitude: None
Tweet started
Retweet status: False
Original tweet: @Hunchback_Whale @ifpfilm @netflix shut up boomer
Created at: 2019-12-03 03:45:33
Clean tweet: Whale shut up boomer
Whale shut up boomer-------------------- 
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
polarity: 0
hasht

Tweet started
Retweet status: False
Original tweet: kevin wendell crumb
Created at: 2019-12-03 03:45:40
Clean tweet: kevin wendell crumb
kevin wendell crumb--------------------- 
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
polarity: 0
hashtags: []
userId: 744992207862697986, userName: yamila, userScreenName: yamibiebersb
user follower count: 1722, userFollowing: False
user friends count: 245
Lets begin to record! All the best :)
latitude: None  logitude: None
Tweet started
Retweet status: False
Original tweet: RT @BBW_BFF: PSA: The Long Kiss Goodnight, one of my favorite movies ever, is on Netflix now. essential viewing during the holidays and any…
Created at: 2019-12-03 03:45:40
Clean tweet: PSA The Long Kiss Goodnight one of my favorite movies ever is on Netflix now essential viewing during the holidays and any
PSA The Long Kiss Goodnight one of my favorite movies ever is on Netflix now essential viewing during the holidays and any 
 {'neg': 0.0, 'neu': 0.714, 'pos': 0.286

KeyboardInterrupt: 

In [ ]:
mydb.close()